## Load dataset

In [21]:
import glob
import pandas as pd

files = glob.glob('../evaluate_results/outer/*.csv', recursive=True)

print(len(files), 'files')
files[0:3]

21 files


["../evaluate_results/outer/KNeighborsClassifier-split_x_y_with_projection-{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60, 80, 100], 'algorithm': ['brute'], 'metric': ['hamming']}-(3 of 5).csv",
 "../evaluate_results/outer/KNeighborsClassifier-split_x_y_normalized_function-{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60, 80, 100], 'algorithm': ['brute'], 'metric': ['hamming']}-(5 of 5).csv",
 "../evaluate_results/outer/KNeighborsClassifier-split_x_y_split_with_one_hot_encoding-{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60, 80, 100], 'algorithm': ['brute'], 'metric': ['hamming']}-(1 of 5).csv"]

In [22]:
# Read all data
data = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])
data.head(2)

,best_params,column,i_outer,metric,model,params,split_method,value
0,"{'algorithm': 'brute', 'metric': 'hamming', 'n...",0,2,accuracy,KNeighborsClassifier,"{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60,...",split_x_y_with_projection,0.296296
1,"{'algorithm': 'brute', 'metric': 'hamming', 'n...",1,2,accuracy,KNeighborsClassifier,"{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60,...",split_x_y_with_projection,0.099537


## Results

In [23]:
result = data.groupby(['model', 'split_method', 'metric'])['value'].agg(["mean", "std"])
result.head()

mean  \
model                split_method                  metric               
KNeighborsClassifier split_x_y                     accuracy  0.356083   
                                                   hit@5     0.538557   
                                                   mdcg      0.549631   
                                                   mrr       0.423612   
                     split_x_y_normalized_function accuracy  0.356083   

                                                                  std  
model                split_method                  metric              
KNeighborsClassifier split_x_y                     accuracy  0.131148  
                                                   hit@5     0.122381  
                                                   mdcg      0.103384  
                                                   mrr       0.128872  
                     split_x_y_normalized_function accuracy  0.131148

## Result as pivot table

In [24]:
result.pivot_table(index=['model', 'split_method'], columns='metric', values=['mean', 'std'])

mean  \
metric                                                      accuracy   
model                split_method                                      
KNeighborsClassifier split_x_y                              0.356083   
                     split_x_y_normalized_function          0.356083   
                     split_x_y_split_with_one_hot_encoding  0.361047   
                     split_x_y_with_projection              0.249185   
                     split_x_y_with_random_matrix           0.249185   

                                                                      \
metric                                                         hit@5   
model                split_method                                      
KNeighborsClassifier split_x_y                              0.538557   
                     split_x_y_normalized_function          0.538557   
                     split_x_y_split_with_one_hot_encoding  0.549269   
                     split_x_y_with_projection              0.430046   
                     split_x_y_with_random_matrix           0.430200   

                                                                      \
metric                                                          mdcg   
model                split_method                                      
KNeighborsClassifier split_x_y                              0.549631   
                     split_x_y_normalized_function          0.549631   
                     split_x_y_split_with_one_hot_encoding  0.559453   
                     split_x_y_with_projection              0.469657   
                     split_x_y_with_random_matrix           0.469513   

                                                                      \
metric                                                           mrr   
model                split_method                                      
KNeighborsClassifier split_x_y                              0.423612   
                     split_x_y_normalized_function          0.423612   
                     split_x_y_split_with_one_hot_encoding  0.435569   
                     split_x_y_with_projection              0.335011   
                     split_x_y_with_random_matrix           0.334886   

                                                                 std  \
metric                                                      accuracy   
model                split_method                                      
KNeighborsClassifier split_x_y                              0.131148   
                     split_x_y_normalized_function          0.131148   
                     split_x_y_split_with_one_hot_encoding  0.157591   
                     split_x_y_with_projection              0.144095   
                     split_x_y_with_random_matrix           0.144095   

                                                                      \
metric                                                         hit@5   
model                split_method                                      
KNeighborsClassifier split_x_y                              0.122381   
                     split_x_y_normalized_function          0.122381   
                     split_x_y_split_with_one_hot_encoding  0.136352   
                     split_x_y_with_projection              0.158261   
                     split_x_y_with_random_matrix           0.158608   

                                                                                
metric                                                          mdcg       mrr  
model                split_method                                               
KNeighborsClassifier split_x_y                              0.103384  0.128872  
                     split_x_y_normalized_function          0.103384  0.128872  
                     split_x_y_split_with_one_hot_encoding  0.117022  0.145402  
                     split_x_y_with_projection              0.122523  0.149018  
                     split_x_y_with